<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a data-toc-modified-id="Шаг-1.-Файл-с-данными-и-общая-информация-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Шаг 1. Файл с данными и общая информация</a></span></li><li><span><a data-toc-modified-id="Шаг-2.-Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Шаг 2. Предобработка данных</a></span><ul class="toc-item"><li><span><a data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a data-toc-modified-id="Шаг-3.-Ответы-на-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Шаг 3. Ответы на вопросы</a></span></li><li><span><a data-toc-modified-id="Шаг-4.-Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Шаг 4. Общий вывод</a></span><ul class="toc-item"><li><span><a data-toc-modified-id="Зависимость-просрочек-по-кредиту-от-количества-детей.-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Зависимость просрочек по кредиту от количества детей.</a></span></li><li><span><a data-toc-modified-id="Зависимость-просрочек-от-семейного-статуса.-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Зависимость просрочек от семейного статуса.</a></span></li><li><span><a data-toc-modified-id="Зависимость-просрочек-от-уровня-доходов.-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Зависимость просрочек от уровня доходов.</a></span></li><li><span><a data-toc-modified-id="Зависимость-просрочек-от-целей-кредита.-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Зависимость просрочек от целей кредита.</a></span></li><li><span><a data-toc-modified-id="Общий-вывод.-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Общий вывод.</a></span></li><li><span><a data-toc-modified-id="Рекомендации.-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Рекомендации.</a></span></li></ul></li></ul></div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Файл с данными и общая информация

In [2]:
import pandas as pd

df = pd.read_csv('/datasets/data.csv')

In [3]:
from pymystem3 import Mystem
from collections import Counter

In [3]:
#Первый взгляд на содержание
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
#Общая информация о всем датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
#Подсчитываем отрицательные значения в стаже
df.loc[df['days_employed'] < 0]['days_employed'].count()

15906

In [6]:
#В столбце со стажем и доходе одинаковое количество пропусков. 
#Проверяем в одних и тех же строках или нет.
df.loc[df['days_employed'].isna()]['total_income'].count()

0

In [7]:
#В обратную сторону для контроля
df.loc[df['total_income'].isna()]['days_employed'].count()

0

In [8]:
#Пример строки с пропуском в стаже и доходе
df.loc[12]

children                           0
days_employed                    NaN
dob_years                         65
education                    среднее
education_id                       1
family_status       гражданский брак
family_status_id                   1
gender                             M
income_type                пенсионер
debt                               0
total_income                     NaN
purpose              сыграть свадьбу
Name: 12, dtype: object

In [9]:
#Посмотрим сколько и каких категорий образования есть в датасете
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [10]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [11]:
#Смотрим какие есть категории социального статуса
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [12]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [13]:
#Проверяем данные о детях
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [14]:
#Проверяем информацию о возрасте
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [15]:
#Проверим сколько есть заемщиков с нулевым возрастом
df.loc[df['dob_years'] == 0]['dob_years'].count()

101

In [16]:
#Проверим данные о поле
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [17]:
#Возможно информация о поле есть в других столбцах?
df.loc[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [18]:
#Проверяем данные об источниках дохода
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [19]:
#Проверяем данные о просрочке
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [20]:
#Проверяем данные о доходах
df.sort_values(by='total_income')['total_income']

14585    20667.263793
13006    21205.280566
16174    21367.648356
1598     21695.101789
14276    21895.614355
             ...     
21489             NaN
21495             NaN
21497             NaN
21502             NaN
21510             NaN
Name: total_income, Length: 21525, dtype: float64

In [21]:
#Проверяем данные о целях кредита
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**

В наборе данных есть пропуски в стаже и доходе. Это одни и те же строки. В целом выглядить как технический сбой или ошибка при заполнении заявки на кредит (возможно неверные формы и т.п.)
Пропуски в стаже не критичны и можно оставить как есть или заполнить средними значениями.
Пропуски в уровне дохода надо будет заполнять медианными значениями, т.к. они учавствуют в целях исследования. Для более глубокого анализа, конечно нужно попытаться заполнить их средними значениями по каким-то категориям, т.к. это 10% от всей выборки.
 
В наборе данных есть неверные значения - например отрицательный стаж в 15906 строках, это 74% выборки. На цели исследования это не влияет, столбец можно оставить как есть, удалить или попытаться построить дни в зависимости от возраста на основе оставшихся данных, но в целом лучше запросить новую выборку у тех кто делал выгрузку из БД, т.к. тут явно техническая ошибка.
 
Есть сопоставление строковых значений категории "образование" числовым категориям, но строковых значений больше, необходимо все приводить к единому описанию и отдельно исследовать на наличие ошибок присвоения верной числовой категории, если влияние образования будет входить в цели исследования
 
Для категории социального статуса количество строковых категорий соответсвует числовым, в целом так же можно проверить на наличие ошибочных присвоений числовых категорий строковым.
 
В колонке "дети" похоже есть аномалии в 20 детей и отрицательные значение, их очень мало относительно всей выборки (окло 0.6%). Можно просто удалить эти строки.
 
В колонке "возраст" есть заемщики с нулевым возрастом. Можно оставить как есть (не влияет на цели исследования) или просто удалить, т.к. их очень малый процент. Если смотреть глубже, то можно посмотреть в какие категории по другим полям они попадают и заменить нули средними возрастами по схожим категориям (средний возраст с определенным уровнем дохода или стажа, например)
 
Один заемщик имеет явно опечатку в графе "Пол" -XNA. В других графах нет информации для гипотез о поле, поэтому можно просто удалить, т.к. одна строка не окажет заметного вличния на результаты исследования.
 
Данные о типе дохода для категорий "безработный, предприниматель, в декрете, студент" в целом не относятся к цели исследованиях, но их явно недостаточно для построения скоринговых систем. В данном случае можно оставить как есть, но для скоринга можно удалить совсем, либо часть приравнять к более крупным категориям. Например, предпринимателя можно приравнять к компаньонам.
 
Данные по наличию задолженности есть у каждого заемщика, столбец не нуждается в корректировках
 
В столбце дохода нужно будет избавиться только от пропусков, нулевых сумм доходов не наблюдается
 
В целях все выглядит разумно: свядьба, недвижимость (жилье, коммерческое), авто, образование. Не видно никаих аномалий. Нужно будет привести к единому виду. Но для целей нашего исследования это не важно.
 
ПС: В дальнейшем, при построении скориноговых матриц, нужно будет учитывать соответсвие связи цели кредита и данных заемщика, и в целом проверять данные на непротиворечивость. Так например земщик состоящий в браке не должен брать кредит на свадьбу и т.п.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [22]:
#Пропуски есть только в полях стажа и дохода
#Стаж не относится к целям исследования, можно заполнить чем угодно, например нулями
#Среднее и медиана не очень подходят, т.к. моного отрицатеьлных значений стажа
#Т.к. значение стажа в днях в дальнейшем можно привести к типу int, поэтому сразу заполним 0
df['days_employed'] = df['days_employed'].fillna(0)

In [23]:
#Пропуски в доходах заполним в зависимости от типов доходов (занятости)
#Проверим сколько пропусков в каждой группе занятости
df.loc[df['total_income'].isna()].groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,147,147,147,147,147,147,147,147,147,0,147
компаньон,508,508,508,508,508,508,508,508,508,0,508
пенсионер,413,413,413,413,413,413,413,413,413,0,413
предприниматель,1,1,1,1,1,1,1,1,1,0,1
сотрудник,1105,1105,1105,1105,1105,1105,1105,1105,1105,0,1105


In [24]:
#Проверим какие характеристики уровня дохода есть по группам занятости
income_type_group_stat = df.groupby('income_type')['total_income'].agg(['count', 'min', 'max', 'mean', 'median'])
income_type_group_stat

,count,min,max,mean,median
income_type,,,,,
безработный,2,59956.991984,2.027225e+05,131339.751676,131339.751676
в декрете,1,53829.130729,5.382913e+04,53829.130729,53829.130729
госслужащий,1312,29200.077193,9.104515e+05,170898.309923,150447.935283
компаньон,4577,28702.812889,2.265604e+06,202417.461462,172357.950966
пенсионер,3443,20667.263793,7.351033e+05,137127.465690,118514.486412
предприниматель,1,499163.144947,4.991631e+05,499163.144947,499163.144947
сотрудник,10014,21367.648356,1.726276e+06,161380.260488,142594.396847
студент,1,98201.625314,9.820163e+04,98201.625314,98201.625314


Заполним пропуски для каждой группы занятости медианными значениями той же группы их основной выборки. Для предпринимателя есть только один заемщик, этого очень мало для обоснования замены, т.к. уровень дохода у предпринимателя можем быть очень разный. Удалять такую важную категорию скорее всего тоже не стоит, вместо этого заполню пропуски для предпринимателя значениями из наиболее близкой группы "компаньон".

In [25]:
#Заполняем пропуски для предпринимателя данными группы "компаньон"
df.loc[(df['income_type'] == 'предприниматель') & df['total_income'].isna(), 'total_income'] = income_type_group_stat.loc['компаньон']['median']

#Создаем список групп занятости для заполнения пропусков
empty_income_type_list = list(df.loc[df['total_income'].isna()].groupby('income_type').count().index)

#Заполняем пропуски медианными значениями по группе
for empty_income in empty_income_type_list:
    df.loc[(df['income_type'] == empty_income) & df['total_income'].isna(), 'total_income'] = income_type_group_stat.loc[empty_income]['median']

#проверяем
df.loc[df['total_income'].isna()].groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,


In [26]:
#Удаляем строку с ошибочным полом
df = df.loc[df['gender'] != 'XNA']
df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

In [27]:
#Удаляем строки с аномальным количеством детей
df = df.loc[(df['children'] >= 0) & (df['children'] < 20)]
df['children'].value_counts()

0    14148
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [28]:
#Переназначем индексы после всех удалений
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21401 entries, 0 to 21400
Data columns (total 12 columns):
children            21401 non-null int64
days_employed       21401 non-null float64
dob_years           21401 non-null int64
education           21401 non-null object
education_id        21401 non-null int64
family_status       21401 non-null object
family_status_id    21401 non-null int64
gender              21401 non-null object
income_type         21401 non-null object
debt                21401 non-null int64
total_income        21401 non-null float64
purpose             21401 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Для целей исследования стаж не важен, поэтому пропуски можно было вообще не заполнять. С другой стороны 74% выборки некорректного стажа наводят на мысль о технической ошибке, возможно стоит перепроверить информацию в БД, если в ней такая же картина, то надо смотреть на процедуру ввода данных с кредитных заявок. Если в БД информация корректная то лучше сделать новую выборку.

Ошибки в столбцах пол и дети имеют малое значение и их проще удалить.

### Замена типа данных

In [29]:
df['days_employed'] = df['days_employed'].astype('int')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21401 entries, 0 to 21400
Data columns (total 12 columns):
children            21401 non-null int64
days_employed       21401 non-null int64
dob_years           21401 non-null int64
education           21401 non-null object
education_id        21401 non-null int64
family_status       21401 non-null object
family_status_id    21401 non-null int64
gender              21401 non-null object
income_type         21401 non-null object
debt                21401 non-null int64
total_income        21401 non-null float64
purpose             21401 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Так как у нас уже имеются числовые значения в столбце стаж, то применяем метод astype, а не to_numeric и переводим из вещественного типа данных который был из-за наличия пропусков в целый

### Обработка дубликатов

In [31]:
#Проверим на наличие дубликатов
df.duplicated().sum()

54

In [32]:
#Пприведём все строковые значения к нижнему регистру, 
#пол можно не приводить, но при обновлении выборки может случиться всякое, поэтому тоже включим
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

In [33]:
#Снова проверим на наличие дубликатов
df.duplicated().sum()

71

In [34]:
#Удаляем полные дубликаты и переустанавливаем индексы
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

**Вывод**

До приведения текстовых полей к нижнему регистру было 54 дубликата, после 71. В целом в этом наборе данных нет каких то уникальных идентификаторов клиента и многие данные вполне могут совпадать, однако маловероятно, что совпадает все, даже формулировка цели кредита, поэтому удаляем все дубликаты.

### Лемматизация

In [4]:
#В ручную составляем список целей кредита исхдя из их описания
m = Mystem()
purpose_list = [] #list(df['purpose'].unique())

for purpose in list(df['purpose'].unique()):
    purpose_list.append(m.lemmatize(purpose))

purpose_list

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

In [36]:
#Создаем категории в зависимости от лемм в целях кредита
#Предполагаем, что если в лемме есть слова "свой", "собсвтенный", то речь идет о жилой недвижимости и личном авто.
#Все операции с недвижимостью без явной ссылки на жилую считаем операциями с коммерческой недвижимостью
#Если есть сово "сдача", то считаем, что это сделка коммерческая

def purpose_category(lemma):
    
    #Обработаем однозначне категории
    
    if 'автомобиль' in lemma: 
        return 'покупка авто'
    elif 'свадьба' in lemma:  
        return 'свадьба'
    elif 'образование' in lemma: 
        return 'образование'

    #Обработаем категории с вариантами
    
    elif 'жилье' in lemma:  #может встречаться несколько вариантов
        if 'ремонт' in lemma:
            return 'ремонт'
        elif 'сдача' in lemma: #цель кредит получение выгоды, а не жилье
            return 'коммерческая недвижимость'
        else:
            return 'жилая недвижимость'
        
    elif 'недвижимость' in lemma:
        if 'жилой' in lemma: #Цель кредита покупка/строительство жилья
            return 'жилая недвижимость'
        if 'собственный' in lemma: #Скорее всего цель кредита покупка/строительство жилья
            return 'жилая недвижимость'
        if 'свой' in lemma: #Скорее всего цель кредита покупка/строительство жилья
            return 'жилая недвижимость'
        else:
            return 'коммерческая недвижимость'
    
    else:
        return 'прочее' #на случай если мы что-то не учли или встретится значение которого не было раньше
        

In [37]:
#Лемматизируем цели кредита
df['purpose_lemmas'] = df['purpose'].apply(m.lemmatize)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [38]:
#Записываем категории вместо лемм
df['purpose_category'] = df['purpose_lemmas'].apply(purpose_category)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",жилая недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",покупка авто
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",жилая недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


In [39]:
#Проверяем, что учли все случаи в функции purpose_category
df.loc[df['purpose_category'] == 'прочее']['purpose_category'].count()

0

**Вывод**

В результате применения лемматизации были выделены шесть целей кредита:
 - получение образования
 - затраты на свадьбу
 - покупка автомобиля
 - ремонт в жилом помещении
 - операции с жилой недвижимостью
 - операции с коммерческой недвижимостью
 
Для сохраненя числа категорий в пределах рекомендованых 5-6, не выделялись в отдельную категорию покупка, продажа или строительство, т.к. были и абстрактные цели "операции с недвижимостью" по которым невозможно устновить планируемое действие и число категорий могло бы быть 8-9, что затруднило бы анализ и размыло бы результаты исследования

### Категоризация данных

In [40]:
#Проверяем базовые характеристики выборки по доходам для создания категорий
income_genral_data = df['total_income'].agg(['count', 'min', 'max', 'mean', 'median'])
income_genral_data

count     2.133000e+04
min       2.066726e+04
max       2.265604e+06
mean      1.653261e+05
median    1.425944e+05
Name: total_income, dtype: float64

Медиана не сильно далеко от среднего значения, значит распределение близко к симметричному.

In [41]:
#Создадим функцию которая возвращает статистические характеристики числовой выборки в виде списка
def  Series_stat(Series):
    return list(Series.agg(['count', 'min', 'max', 'mean', 'median']))

In [42]:
#Разобьем всю совокупность на 10 групп и посмотрим на характеристики каждой группы
groupe_count = 10

step = int((income_genral_data['max'] - income_genral_data['min'] + 1) / groupe_count) # +1 чтобы последний интервал был больше "max"

statistic_data = []

for i in range(groupe_count):
    
    #формирем статистические характеристики для группы
    step_statistic_data = Series_stat(df.loc[(df['total_income'] >= ((i * step) + income_genral_data['min'])) & (df['total_income'] < (((i + 1) * step) + income_genral_data['min']))]['total_income'])
    
    print(step_statistic_data)

[18392.0, 20667.26379327158, 245153.7095083065, 136970.4662429369, 137945.58683199977]
[2652.0, 245172.8971950128, 469637.1911006271, 309640.43854667584, 291408.70391545014]
[223.0, 471008.5165567714, 694036.2278389871, 548442.3010932556, 527452.8332531311]
[34.0, 694150.8908757226, 910451.4708141336, 778077.1985238788, 758169.3378911191]
[15.0, 920897.7180211036, 1128836.2562180862, 1038343.6015294491, 1030899.173447853]
[6.0, 1172459.831538252, 1350245.6054374685, 1258469.4208073083, 1259393.8046527326]
[2.0, 1427934.4631516722, 1551152.8938321068, 1489543.6784918895, 1489543.6784918895]
[4.0, 1597613.490466747, 1726276.0143316737, 1687554.2913263047, 1713163.8302533985]
[0.0, nan, nan, nan, nan]
[1.0, 2200852.210258896, 2200852.210258896, 2200852.210258896, 2200852.210258896]


Большая часть заемщиков имеет доход от 20 до 245 тысяч. Все что выше 245000 будем считать как "высокий" доход.

In [43]:
#Разделим выборку с доходом до 245 тыс еще на 10 частей.
groupe_count = 10

df_first_group = df.loc[df['total_income'] < 245000]
income_first_group_data = df_first_group['total_income'].agg(['count', 'min', 'max', 'mean', 'median'])


step = int((income_first_group_data['max'] - income_first_group_data['min'] + 1) / groupe_count) # +1 чтобы последний интервал был больше "max"

statistic_data = []

for i in range(groupe_count):
    
    #формирем статистические характеристики для группы
    step_statistic_data = Series_stat(df_first_group.loc[(df_first_group['total_income'] >= ((i * step) + income_first_group_data['min'])) & (df_first_group['total_income'] < (((i + 1) * step) + income_first_group_data['min']))]['total_income'])
    
    print(step_statistic_data)

[183.0, 20667.26379327158, 43067.05995680563, 36414.761425771685, 37670.08922714804]
[948.0, 43128.98188731297, 65520.432920286075, 56358.6408113977, 56849.54815842959]
[1997.0, 65533.416063870995, 87961.00041568266, 77776.95377108165, 78661.12018118]
[2544.0, 87979.4053811268, 110392.92530251283, 99473.68253168852, 99625.78141603121]
[2946.0, 110396.61025124694, 132826.80362604655, 121003.29722805179, 119552.31845512846]
[3567.0, 132836.62974644848, 155248.18475389478, 143522.18921832426, 142594.39684740017]
[2590.0, 155270.6363260297, 177674.95003137094, 167136.5031205336, 168211.22224837553]
[1530.0, 177692.47622608262, 200116.89854182387, 188906.15861024053, 188926.33253914025]
[1160.0, 200178.36266429359, 222538.37227671413, 210792.6236266448, 210654.89953878545]
[918.0, 222573.04099661933, 244957.2622338864, 232715.43252538596, 231951.2166541234]


Распределение более менее симметричное. Если в группы с высоким доходом попало около 3 тыс заемщиков, то для группы "низкий" возьмем потребителей с доходом до 88 тыс.
В итоге получим три группы:
 - низкий, доход до 88 тыс
 - средний, доход от 88 до 245 тыс
 - высокий, доход свыше 245

In [44]:
#Функция категоризации по доходу
def income_category(income):
    if income < 88000:
        return 'низкий'
    elif income > 245000:
        return 'высокий'
    else:
        return 'средний'

In [45]:
#Добавляем столбец с категорией дохода
df['income_category'] = df['total_income'].apply(income_category)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_category,income_category
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",жилая недвижимость,высокий
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",покупка авто,средний
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",жилая недвижимость,средний
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,средний


In [46]:
df.groupby('income_category')['income_category'].count()

income_category
высокий     2946
низкий      3131
средний    15253
Name: income_category, dtype: int64

**Вывод**

В целом выборка близка к симметричной с медианой около 145 тысяч и средним около 165 тысяч.
Минимальный и максимальный доход 20 тысяч и 2,26 млн соответственно.
85% заемщиков (около 18 тысяч) имеют доход до 254 тысяч рублей.

В итоге получим три группы:
 - низкий, доход до 88 тыс имеют 3131 заемщика
 - средний, доход от 100 до 245 тыс имеют 15253 заемщика
 - высокий, доход свыше 245 имеют 2946 заемщиков
 
Для более детального анализа можно было бы ввести больше градаций по уровню дохода и дополнительно проводить анализ на соответствие целей кредита.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [47]:
#Проверяем заемщиков без детей и наличие просрочек
debt_without_children = df.loc[df['children'] == 0].groupby('debt')['debt'].count()

#Считаем процент имеющих просрочки из общего числа заемщиков без детей
debt_without_children[1] / (debt_without_children[0] + debt_without_children[1]) * 100

7.544357700496807

In [48]:
#Проверяем сколько заемщиков с детми имеют просрочки
debt_with_children = df.loc[df['children'] != 0].groupby('debt')['debt'].count()

#Считаем процент имеющих просрочки из общего числа заемщиков c детьми
debt_with_children[1] / (debt_with_children[0] + debt_with_children[1]) * 100

9.240331491712707

In [49]:
#Проверим меняется ли зависимость от количества детей
df.groupby('children')['debt'].value_counts()

children  debt
0         0       13027
          1        1063
1         0        4364
          1         444
2         0        1858
          1         194
3         0         303
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64

**Вывод**

В целом видно, что прострочка по кредитам встречается в 8-10% случаях при любом количестве детей, что соответсвует проценту просрочек по всей выборке. Не имеют просрочки 19784 заемщика, имеют просрочку 1741 заемщика.

Наличие детей не оказывает заметного влияния на просрочку по кредиту.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [50]:
#Проверим распределение просрочек в зависимости от семейного статуса
family_status_debt = df.groupby('family_status')['debt'].value_counts()
family_status_debt

family_status          debt
в разводе              0        1105
                       1          84
вдовец / вдова         0         888
                       1          63
гражданский брак       0        3748
                       1         385
женат / замужем        0       11334
                       1         927
не женат / не замужем  0        2523
                       1         273
Name: debt, dtype: int64

In [51]:
#Подсчитаем процент должников для каждой категории семейного статуса
family_status_list = list(df['family_status'].unique())
for status in family_status_list:
    print(status, '------->', family_status_debt[status][1] / (family_status_debt[status][0] + family_status_debt[status][1]) * 100)

женат / замужем -------> 7.560557866405676
гражданский брак -------> 9.31526736027099
вдовец / вдова -------> 6.624605678233439
в разводе -------> 7.064760302775442
не женат / не замужем -------> 9.763948497854077


**Вывод**

Процент должников в разных категориях семейного статуса колеблится от 6,6 до 9,7, что соответсвует общему проценту должников по выборке в целом.
Вдовцы в целом имеют меньше просроченных кредитов, чем живущие в гражданском браке, но относительно всей выборки все остается близко к общим значениям.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [52]:
#Проверим распределение просрочек в зависимости от уровня дохода
income_category_debt = df.groupby('income_category')['debt'].value_counts()
income_category_debt

income_category  debt
высокий          0        2742
                 1         204
низкий           0        2891
                 1         240
средний          0       13965
                 1        1288
Name: debt, dtype: int64

In [53]:
#Подсчитаем процент должников для каждой категории дохода
income_category_list = list(df['income_category'].unique())
for category in income_category_list:
    print(category, '----->', income_category_debt[category][1] / (income_category_debt[category][0] + income_category_debt[category][1]) * 100)

высокий -----> 6.924643584521386
средний -----> 8.444240477283158
низкий -----> 7.665282657297988


**Вывод**

Процент должников в каждой категории по уровню дохода соответсвует общему проценту по всей выборке.

Уровень дохода не оказывает заметного влияния на наличие просрочки по кредиту.

- Как разные цели кредита влияют на его возврат в срок?

In [54]:
#Проверим распределение просрочке в зависимости от целей
purpose_category_debt = df.groupby('purpose_category')['debt'].value_counts()
purpose_category_debt

purpose_category           debt
жилая недвижимость         0       5256
                           1        401
коммерческая недвижимость  0       4145
                           1        344
образование                0       3619
                           1        369
покупка авто               0       3879
                           1        400
ремонт                     0        569
                           1         35
свадьба                    0       2130
                           1        183
Name: debt, dtype: int64

In [55]:
#Подсчитаем процент должников для каждой категории дохода
purpose_list = list(df['purpose_category'].unique())

for purpose in purpose_list:
    print(purpose, '----->', purpose_category_debt[purpose][1] / (purpose_category_debt[purpose][0] + purpose_category_debt[purpose][1]) * 100)

жилая недвижимость -----> 7.088562842496023
покупка авто -----> 9.34797849964945
образование -----> 9.252758274824474
свадьба -----> 7.911802853437094
коммерческая недвижимость -----> 7.663176654043216
ремонт -----> 5.7947019867549665


**Вывод**

Немного выделяется цель кредита - ремонт, но выборка данной категории около 2,5% от всех заемщиков, возможно просто статистические колебания. Никаких существенных отличий от средних показателей всей выборки нет.

Цель кредита не оказывает заметного влияния на наличие прострочки по кредиту.

## Шаг 4. Общий вывод

  В наборе данных есть пропуски в стаже и доходе. Это одни и те же строки, которые составляют 10% всей выборки. 
  
  В стаже так же 15906 записей из 21524 имеют отрицательные значения. Для целей данного исследования стаж не важен, поэтому недостающие значения были заполнены нулями. С другой стороны, с точки зрения построения скоринговой системы стаж является важным фактором оценки заемщика, 74% выборки некорректного стажа наводят на мысль о технической ошибке, возможно стоит перепроверить информацию в БД, если в ней такая же картина, то надо смотреть на процедуру ввода данных с кредитных заявок. Если в БД информация корректная то лучше сделать новую выборку.

  Уровнь дохода учавствует в целях исследования, поэтому пропуски были заполнены медианными значениями уровня дохода соответствующей группы занятости основной выборки. Для предпринимателя есть только один заемщик, этого очень мало для обоснования замены, т.к. уровень дохода у предпринимателя можем быть очень разный. Удалять такую важную категорию скорее всего тоже не стоит, вместо этого  пропуск для предпринимателя был заменён значением из наиболее близкой группы "компаньон".
 
  В выборке обнаружено несоответствие строковых значений категории "образование" числовым категориям, строковых значений больше. Необходимо все приводить к единому описанию и отдельно исследовать на наличие ошибок присвоения верной числовой категории, если влияние образования будет входить в скоринговую модель.
 
  В колонке "дети" есть аномалии в 20 детей и отрицательные значение, их очень мало относительно всей выборки (окло 0.6%), поэтому аномальные строки были удалены.
 
  В колонке "возраст" есть заемщики с нулевым возрастом, что опять же может указывать либо на технические ошибки в БД или при выгрузке или на ошибки при заполнении кредитных заявок. 
 
  Один заемщик имеет явно опечатку в графе "Пол" -XNA. В других графах нет информации для гипотез о поле, поэтому он был удален, т.к. одна строка не окажет заметного вличния на результаты исследования.
 
Данные о типе дохода для категорий "безработный, предприниматель, в декрете, студент" в целом не относятся к цели исследования, но их явно недостаточно для построения скоринговых систем. В данном случае можно оставить как есть, но для скоринга можно удалить совсем, либо часть приравнять к более крупным категориям. Например, предпринимателя можно приравнять к компаньонам.
 
Данные по наличию задолженности есть у каждого заемщика, столбец не нуждается в корректировках
 
В целях кредита все выглядит разумно: свядьба, недвижимость (жилье, коммерческое), авто, образование. Не видно никаих аномалий. В результате применения лемматизации были выделены шесть целей кредита:
 - получение образования
 - затраты на свадьбу
 - покупка автомобиля
 - ремонт в жилом помещении
 - операции с жилой недвижимостью
 - операции с коммерческой недвижимостью
 
Для сохраненя числа категорий в пределах рекомендованых 5-6, не выделялись в отдельную категорию покупка, продажа или строительство, т.к. были и абстрактные цели "операции с недвижимостью" по которым невозможно устновить планируемое действие и число категорий могло бы быть 8-9, что затруднило бы анализ и размыло бы результаты исследования
 
В дальнейшем, при построении скориноговых матриц, нужно будет учитывать соответсвие связи цели кредита и данных заемщика, и в целом проверять данные на непротиворечивость. Так например земщик состоящий в браке не должен брать кредит на свадьбу и т.п.

По уровню дохода выборка близка к симметричной с медианой около 145 тысяч и средним около 165 тысяч. Минимальный и максимальный доход 20 тысяч и 2,26 млн соответственно. 85% заемщиков (около 18 тысяч) имеют доход до 254 тысяч рублей. Для целей исследования были выделены следующие группы по уровню дохода:
 - низкий, доход до 88 тыс имеют 3131 заемщика
 - средний, доход от 100 до 245 тыс имеют 15253 заемщика
 - высокий, доход свыше 245 имеют 2946 заемщиков
 
<b>В целом по выборке имеют просрочку 1741 заемщика из 21524, что составляет 8% заемщиков.</b>

### Зависимость просрочек по кредиту от количества детей.
    
  Прострочка по кредитам встречается в 8-10% случаях при любом количестве детей, что соответсвует проценту просрочек по всей выборке. Наличие детей не оказывает заметного влияния на просрочку по кредиту.
    
### Зависимость просрочек от семейного статуса.

  Процент должников в разных категориях семейного статуса меняется от 6,6 до 9,7. Вдовцы имеют меньше просроченных кредитов, чем живущие в гражданском браке, но относительно всей выборки все остается близко к общим значениям.
    
### Зависимость просрочек от уровня доходов.

  Процент должников в каждой категории по уровню дохода соответсвует общему проценту по всей выборке. Уровень дохода не оказывает заметного влияния на наличие просрочки по кредиту.
    
### Зависимость просрочек от целей кредита.

  Немного выделяется цель кредита - ремонт в 5,8%, но выборка данной категории около 2,5% от всех заемщиков, возможно просто статистические колебания. Никаких существенных отличий от средних показателей всей выборки нет. Цель кредита не оказывает заметного влияния на наличие прострочки по кредиту.
    
### Общий вывод.

  На наличие просточки по кредиту не оказывает существенного влияния ни семейное положение, ни наличие детей, ни источники дохода, ни цель кредита. Процент просрочек остается практически равным общему проценкут по всей выборке, около 8%.
    
### Рекомендации.

  Большой процент отрицательного стажа, заметный процент пропусков в стаже и уровне дохода, а также наличие нулевого возраста и аномальных значений количества детей заемщика говорят о вероятной технической ошибке или ошибках заполнения кредитных заявок. Рекомендуется проверить информацию по данным заемщиков в БД, процедуру выгрузки из БД, а так же процедуру заполнения и проверки кредитных заявок заемщиками.
  Так же стоит привести в соответствие друг другу текстовые и числовые категории образования.